In [1]:
import geoio
import os
from glob import glob
import numpy  as np
import gdal
from tifffile import imsave

In [2]:
filename = os.getcwd()+"\\stacked_images\\"
month_list = glob(filename+"*")
month_list

['C:\\Users\\Hello\\Documents\\remote-sensing\\APIs\\SentinelSat\\stacked_images\\dec_data',
 'C:\\Users\\Hello\\Documents\\remote-sensing\\APIs\\SentinelSat\\stacked_images\\jan_data']

In [3]:
#curating files w.r.t date
paths = {}
for month in month_list:
    stacks = glob(month+"\\*.tif")
    for stack in stacks:
        date = int(stack.split("\\")[-1].split('.')[0].split('_')[-1][:8])
        value = []
        if date in paths.keys():
            value = paths.get(date)
        value.append(stack)
        paths.update({date: value})

In [4]:
#filter by tiles count
maxTiles = int(input("How many tiles to be merged? "))
pop = []
for key, value in paths.items():
    if len(value) < maxTiles:
        pop.append(key)
for key in pop:
    paths.pop(key)

How many tiles to be merged? 5


In [5]:
def getStackedArray(file_path):
    print(file_path.split('\\')[-1])
    ds = gdal.Open(file_path)
    nir = np.array(ds.GetRasterBand(1).ReadAsArray())
    blue = np.array(ds.GetRasterBand(2).ReadAsArray())
    green = np.array(ds.GetRasterBand(3).ReadAsArray())
    red = np.array(ds.GetRasterBand(4).ReadAsArray())
    image = np.dstack((nir, blue, green, red))
    print(red)
    return image

In [6]:
#delete improper scenes
for key, value in paths.items():
    if len(value) > maxTiles:
        print("There are extra tiles. Creating the best match " + str(key) + "... Recheck the mosaic for this")
        coordinates = []
        for path in value:
            img = geoio.GeoImage(path)
            pos = img.raster_to_proj(0,0)
            coordinates.append(pos)
        commons = {}
        i = 0
        for coordinate in coordinates:
            ls = []
            if coordinate in commons.keys():
                ls = commons.get(coordinate)
            ls.append(value[i])
            i = i + 1
            commons.update({coordinate: ls})
        for key in commons.keys():
            files = commons[key]
            tiles = len(files)
            stacked_arrays = []
            for image_path in files:
                stacked_arrays.append(getStackedArray(image_path))
            if tiles  >  1:
                black = np.zeros(4, dtype = int)
                for i in range(stacked_arrays[0].shape[0]):
                    for j in range(stacked_arrays[0].shape[1]):
                        if np.array_equal(stacked_arrays[0][i][j], black):
                            next_image = 1
                            while(next_image < tiles):
                                print(str(i) + " " + str(j))
                                if not np.array_equal(stacked_arrays[next_image][i][j], black):
                                    np.copyto(stacked_arrays[0][i][j], stacked_arrays[next_image][i][j])
                                next_image = next_image + 1
            imsave("tile.tif", stacked_arrays[0])
            print("saved...")

There are extra tiles. Creating the best match 20190106... Recheck the mosaic for this
S2B_MSIL2A_20190106T050209_N0211_R119_T44PLC_20190106T101624.tif
[[   0    0    0 ... 1274 1260 1260]
 [   0    0    0 ... 1274 1294 1288]
 [   0    0    0 ... 1278 1302 1266]
 ...
 [   0    0    0 ... 1542 1538 1510]
 [   0    0    0 ... 1522 1504 1490]
 [   0    0    0 ... 1528 1516 1476]]
saved...
S2B_MSIL2A_20190106T050209_N0211_R119_T44PMC_20190106T093021.tif
[[1430 1426 1462 ... 1112 1134 1142]
 [1414 1406 1414 ... 1168 1154 1152]
 [1414 1420 1428 ... 1166 1110 1120]
 ...
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]]
S2B_MSIL2A_20190106T050209_N0211_R119_T44PMC_20190106T101624.tif


KeyboardInterrupt: 